In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries
from skforecast.model_selection_multiseries import random_search_forecaster_multiseries
from skforecast.model_selection_multiseries import bayesian_search_forecaster_multiseries

# Se transforman los datos, se crean variables exogenas y se coloca la columna datetime como indice

In [25]:
df = pd.read_csv("df_time_serie_2022.csv")
df.FECHA = pd.to_datetime(df.FECHA)
df.set_index("FECHA", inplace=True)


# Se eliminan las columnas de más, de igual manera se eliminan los productos con menos de 50 ventas en 2023

In [26]:
df["Dia de la semana"] = df.index.dayofweek

In [29]:
exog = df[["Dia de la semana","Mes", "Dia", "Hora"]]
df.drop(columns=["Dia de la semana","Mes", "Dia", "Hora"], inplace=True)

In [ ]:
#Agregar dia del mes


exog["Dia del mes"] = exog.index.day
#Convertir a dummies dia de la semana y mes
exog = pd.get_dummies(exog, columns=["Dia de la semana","Mes"])

In [ ]:
#Convert the days and hour to cos
exog["hour_cos"] = np.cos(2*np.pi*exog["Hora"]/24)
exog["day_cos"] = np.cos(2*np.pi*exog["Dia del mes"]/31)
exog["Dia del año"] = np.cos(2*np.pi*exog["Dia"]/7)
#drop
exog.drop(columns=["Hora","Dia del mes","Dia"], inplace=True)
exog

In [ ]:
df = df.asfreq("h")

Crear Exog asfreq h sin valores nulos, cambiar tipo de valor de bool a int

In [34]:
exog = exog.asfreq("h")
#Convert bool to int
exog = exog.astype(int)

In [36]:
# Seleccionar las columnas que contengan la palabra "SURESTE" y no "SURESTE 2" o "SURESTE 3"
x = df.filter(regex=("SURESTE(?! 2| 3)")).columns

# Se divide el test en validacion, test y entrenamiento

In [ ]:
# Split data into train-validation-test
# ======================================================================================
end_train = '2024-02-29'
end_val = '2024-07-30'

data_train = df.loc[:end_train, :].copy()
data_val   = df.loc[end_train:end_val, :].copy()
data_test  = df.loc[end_val:, :].copy()
exog_train = exog.loc[:end_train, :].copy()
exog_val   = exog.loc[end_train:end_val, :].copy()
exog_test  = exog.loc[end_val:, :].copy()

print(f"Train dates      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Validation dates : {data_val.index.min()} --- {data_val.index.max()}  (n={len(data_val)})")
print(f"Test dates       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

Medidas a cambiar a futuro:
Tiempo a predecir: 6 horas adelante, 12 horas adelante y un día.

ForecasterAutoregMultiVariate, con todo el dataframe, con una estación.

Algunas de los trends que hay

## Se buscan los mejores hiperparametros para el modelo por grid-search y se guardan en un dataframe

Intento con datos de una estación, solo 6 horas

In [ ]:
# Bayesian search hyperparameters and lags with Optuna
# ==============================================================================
forecaster = ForecasterAutoregMultiVariate(
                 regressor = LGBMRegressor(random_state=123, verbose=-1),
                 level     = 'SURESTE PM2.5 (UG/M3)',
                 lags      = 24,
                 steps     = 6
             )

# Search space
def search_space(trial):
    search_space  = {
        'lags'            : trial.suggest_categorical('lags', [7, 14]),
        'n_estimators'    : trial.suggest_int('n_estimators', 10, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1., 10),
        'max_features'    : trial.suggest_categorical('max_features', ['log2', 'sqrt'])
    }

    return search_space

results, best_trial = bayesian_search_forecaster_multiseries(
                          forecaster            = forecaster,
                          series                = df[x],
                          exog                  = exog, 
                          search_space          = search_space,
                          steps                 = 6,
                          metric                = ['mean_absolute_error', 'mean_squared_error', 'mean_absolute_percentage_error' ],
                          aggregate_metric      = 'weighted_average',
                          refit                 = False,
                          initial_train_size    = len(data_train),
                          fixed_train_size      = False,
                          n_trials              = 5,
                          random_state          = 123,
                          return_best           = False,
                          n_jobs                = 'auto',
                          verbose               = True,
                          show_progress         = True,
                          engine                = 'optuna',
                          kwargs_create_study   = {},
                          kwargs_study_optimize = {}
                      )
results.head(4)

# Forecaster multi-variate to model each region together

In [56]:
sureste = df.filter(regex=("SURESTE")).columns

In [ ]:
sureste

In [ ]:
# Bayesian search hyperparameters and lags with Optuna
# ==============================================================================
forecaster = ForecasterAutoregMultiVariate(
                 regressor = LGBMRegressor(random_state=123, verbose=-1),
                 level     = 'SURESTE PM2.5 (UG/M3)',
                 lags      = 24,
                 steps     = 6
             )

# Search space
def search_space(trial):
    search_space  = {
        'lags'            : trial.suggest_categorical('lags', [7, 14]),
        'n_estimators'    : trial.suggest_int('n_estimators', 10, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1., 10),
        'max_features'    : trial.suggest_categorical('max_features', ['log2', 'sqrt'])
    }

    return search_space

results, best_trial = bayesian_search_forecaster_multiseries(
                          forecaster            = forecaster,
                          series                = df[sureste],
                          exog                  = exog, 
                          search_space          = search_space,
                          steps                 = 6,
                          metric                = ['mean_absolute_error', 'mean_squared_error', 'mean_absolute_percentage_error' ],
                          aggregate_metric      = 'weighted_average',
                          refit                 = False,
                          initial_train_size    = len(data_train),
                          fixed_train_size      = False,
                          n_trials              = 5,
                          random_state          = 123,
                          return_best           = False,
                          n_jobs                = 'auto',
                          verbose               = True,
                          show_progress         = True,
                          engine                = 'optuna',
                          kwargs_create_study   = {},
                          kwargs_study_optimize = {}
                      )
results.head(4)

Obtener una tabla con R^2,  MSE, RSME

# Se hará un modelo por region

Se puede guardar cada modelo en un csv, que puede ser releido para usarse en el futuro, pero es recomendable reentrenar cada mes

Se crea un dataframe con cada uno de los parametros necesarios

In [89]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score

Se entrena cada modelo y se predice para una semana y para un mes, se guarda en un dataframe cada predicción

In [68]:
#Get each first part of the column name
regions = df.columns.str.split(" ").str[0].unique()

In [ ]:
for region in regions:
    # Create and fit forecaster MultiVariate
    # ==============================================================================
    forecaster = ForecasterAutoregMultiVariate(
                    regressor          = LGBMRegressor(random_state=123, verbose=-1),
                    level              = 'SURESTE PM2.5 (UG/M3)', #Hay que hacer que cambie de SURESTE a SURESTE 2, 3 y cada region tiene sus propios numeros
                    lags               = 21,
                    steps              = 6,
                    transformer_series = StandardScaler(),
                    transformer_exog   = None,
                    weight_func        = None,
                    n_jobs             = 'auto'
                    min_samples_leaf   = 9,
                    n_estimators       = 10,
                    max_features       = 'log2'
                    
                )

    forecaster.fit(series=data_train.filter(regex=(region)), exog=exog_train)
    predictions = forecaster.predict(steps=6, last_window=data_val.filter(regex=(region)), exog=exog_val)
    

In [87]:
    # Create and fit forecaster MultiVariate
    # ==============================================================================
forecaster = ForecasterAutoregMultiVariate(
                regressor          = LGBMRegressor(random_state=123, verbose=-1),
                level              = 'SURESTE PM2.5 (UG/M3)', #Hay que hacer que cambie de SURESTE a SURESTE 2, 3 y cada region tiene sus propios numeros
                lags               = 21,
                steps              = 6,
                transformer_series = StandardScaler(),
                transformer_exog   = None,
                weight_func        = None,
                n_jobs             = 'auto',
                    
            )

forecaster.fit(series=data_train.filter(regex=("SURESTE")))


In [88]:
predictions = forecaster.predict(steps=6, last_window=data_val.filter(regex=("SURESTE")))


Puede predecir 6 horas

2024-07-31 05:00:00

In [ ]:
predictions

In [ ]:
# Get the models stats
data_test.filter(regex=("SURESTE PM2.5")).loc["2024-07-31"].head(6)

In [ ]:
# Get the R^2, RMSE, MAE and MAPE
r2 = r2_score(data_test.filter(regex=("SURESTE PM2.5")).loc["2024-07-31"].head(6), predictions)
rmse = mean_squared_error(data_test.filter(regex=("SURESTE PM2.5")).loc["2024-07-31"].head(6), predictions, squared=False)
mae = mean_absolute_error(data_test.filter(regex=("SURESTE PM2.5")).loc["2024-07-31"].head(6), predictions)
mape = mean_absolute_percentage_error(data_test.filter(regex=("SURESTE PM2.5")).loc["2024-07-31"].head(6), predictions)

print(f"R^2: {r2}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}")


In [ ]:
# Plot the predicted, vs real
plt.plot(data_test.filter(regex=("SURESTE PM2.5")).loc["2024-07-31"].head(6), label="Real")
plt.plot(predictions, label="Predicted")
plt.legend()

# Modelo con datos de una sola estacion

In [113]:
    # Create and fit forecaster MultiVariate
    # ==============================================================================
forecaster = ForecasterAutoregMultiVariate(
                regressor          = LGBMRegressor(random_state=123, verbose=-1),
                level              = 'SURESTE PM2.5 (UG/M3)', #Hay que hacer que cambie de SURESTE a SURESTE 2, 3 y cada region tiene sus propios numeros
                lags               = 21,
                steps              = 6,
                transformer_series = StandardScaler(),
                transformer_exog   = None,
                weight_func        = None,
                n_jobs             = 'auto',
                    
            )

forecaster.fit(df.filter(regex=("SURESTE(?! 2| 3)")).loc[:end_train])

In [ ]:
predictions = forecaster.predict(steps=6, last_window=df.filter(regex=("SURESTE(?! 2| 3)")).loc[end_train:end_val])
predictions


In [ ]:
# Get the R^2, RMSE, MAE and MAPE
r2 = r2_score(data_test.filter(regex=("SURESTE PM2.5")).loc["2024-07-31"].head(6), predictions)
rmse = mean_squared_error(data_test.filter(regex=("SURESTE PM2.5")).loc["2024-07-31"].head(6), predictions, squared=False)
mae = mean_absolute_error(data_test.filter(regex=("SURESTE PM2.5")).loc["2024-07-31"].head(6), predictions)
mape = mean_absolute_percentage_error(data_test.filter(regex=("SURESTE PM2.5")).loc["2024-07-31"].head(6), predictions)

print(f"R^2: {r2}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}")

In [ ]:
# Plot the predicted, vs real
plt.plot(data_test.filter(regex=("SURESTE PM2.5")).loc["2024-07-31"].head(6), label="Real")
plt.plot(predictions, label="Predicted")
plt.legend()

In [32]:
for i in models.index:
    forecaster = ForecasterAutoreg(
                     regressor     = HistGradientBoostingRegressor(random_state=123, max_depth=models.loc[i,"max_depth"], max_iter=models.loc[i,"max_iter"], learning_rate=models.loc[i,"learning_rate"]),
                     lags          = [models.loc[i,"lags"].max()],
                     transformer_y = StandardScaler()
                 )
    forecaster.fit(y=data.loc[:"2023-11-30",i], exog=exog.loc[:"2023-11-30", :])
    month_pred[i] = forecaster.predict(steps=31, exog=exog.loc["2023-12-01":])
    week_pred[i] = forecaster.predict(steps=8, exog=exog.loc["2023-12-01":])

Se ayuda un poco al modelo dado que los valores negativos no existen en los productos

Se consiguen las estadisticas que tiene el modelo para predecir durante un mes y durante una semana

In [ ]:
stats_mes = pd.DataFrame()
for i in month_pred.columns:
    y_pred = month_pred[i]
    y_test = data.loc["2023-12-01":"2023-12-31",i]
    mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
    mape = mean_absolute_percentage_error(y_true=y_test, y_pred=y_pred)
    mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
    r2 = r2_score(y_true=y_test, y_pred=y_pred)
    mape2 = abs((y_pred - y_test)/y_test).replace([np.inf, -np.inf], np.log(0.9999999999999999999999999)).dropna().sum()/30
    mape3 = (np.abs((y_test - y_pred)/np.where(y_test==0, 1, y_test))).mean()
    smape = 1/len(y_test) * np.sum(2*np.abs(y_pred - y_test)/(np.abs(y_pred) + np.abs(y_test))*100)
    valor_real = y_test.sum()
    valor_pred = y_pred.sum()
    error = (valor_real - valor_pred)/valor_real
    error_semanal = error/4 * 100
    stats_mes = pd.concat([stats_mes, pd.DataFrame({"Producto":i, "MAE":mae, "MSE":mse, "R2":r2, "SMAPE": smape,"MAPE lib":mape,"MAPE2":mape2,"MAPE3":mape3, "valor real": 
        valor_real, "valor predecido": valor_pred, "error":error*100, "error por semana": error_semanal}, index=[0])], axis=0)
stats_mes

In [ ]:
stats_semana = pd.DataFrame()
for i in month_pred.columns:
    y_pred = week_pred[i]
    y_test = data.loc["2023-12-01":"2023-12-8",i]
    mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
    mape = mean_absolute_percentage_error(y_true=y_test, y_pred=y_pred)
    mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
    r2 = r2_score(y_true=y_test, y_pred=y_pred)
    mape2 = abs((y_pred - y_test)/y_test).replace([np.inf, -np.inf], np.log(0.9999999999999999999999999)).dropna().sum()/30
    mape3 = (np.abs((y_test - y_pred)/np.where(y_test==0, 1, y_test))).mean()
    smape = 1/len(y_test) * np.sum(2*np.abs(y_pred - y_test)/(np.abs(y_pred) + np.abs(y_test))*100)
    valor_real = y_test.sum()
    valor_pred = y_pred.sum()
    error = (valor_real - valor_pred)/valor_real *100
    stats_semana = pd.concat([stats_semana, pd.DataFrame({"Producto":i, "MAE":mae, "MSE":mse, "R2":r2, "SMAPE": smape,"MAPE lib":mape,"MAPE2":mape2,"MAPE3":mape3, "valor real": 
        valor_real, "valor predecido": valor_pred, "error":error}, index=[0])], axis=0)
stats_semana

In [ ]:
#Save figure#
for i in month_pred.columns:
    fig, ax=plt.subplots(figsize=(7, 3))
    month_pred[i].plot(ax=ax,color='red', linestyle='--', label='Predicciones')
    data.loc["2023-12-01":"2023-12-31",i].plot(ax=ax,color='blue', linestyle='-', label='Valores reales')
    ax.set_title(i)
    ax.legend()
    ax.set_ylabel('Ventas')
    ax.set_xlabel('Fecha')
    #plt.savefig(f"pred_mensual_{i}.png")
    

## Agregar modelo del otro producto y comparar

In [ ]:
for i in week_pred.columns:
    fig, ax=plt.subplots(figsize=(7, 3))
    week_pred[i].plot(ax=ax,color='#ECA24E', linestyle='--', label='Predicciones', linewidth=2.5)
    data.loc["2023-12-01":"2023-12-08",i].plot(ax=ax,color='#1f93cf', linestyle='-', label='Valores reales', linewidth=2.5)
    ax.set_title(i, color = 'white')
    ax.set_facecolor('#0e4588')
    ax.grid(color='white')
    #ax.tick_params(axis='x', colors='white')
    #ax.spines['bottom'].set_color('white')
    #ax.spines['top'].set_color('white')
    #ax.xaxis.label.set_color('white')
    #ax.tick_params(axis='x', colors='white', labelsize = 10, which = 'major')
    fig.set_facecolor('#1f93cf')
    ax.set_ylabel('Ventas',color='white') 
    ax.set_xlabel('Fecha',color='white')
    plt.xticks(color='white', weight='bold')
    plt.yticks(color='white', weight='bold')
    #plt.legend(color='white')
    ax.legend()

In [ ]:
resid = data["Producto 0"].loc["2023-10-21":"2023-10-28"] - predicts

In [ ]:
week_pred.to_csv("predicciones_semanales_HGB.csv")
month_pred.to_csv("predicciones_mensuales_HGB.csv")
data.to_csv("datatop20.csv")